In [16]:
import PyPDF2
from PIL import Image
import os
import sys

# Definindo constantes 

PDF_MASTER = 'Master_D.pdf'
PDF_DOCUMENTO1 = 'Documento1.pdf'
IMAGE_INPUT = 'master_img.png'
PDF_MERGED_OUTPUT = 'Merge.pdf'
PDF_IMAGE_OUTPUT = 'imagem.pdf'


def extract_and_analyze_pdf(input_pdf_path):
    """
     O conteúdo de um PDF, conta páginas e palavras, e salva cada página em um arquivo de texto.

    Args:
        input_pdf_path (str): O caminho para o arquivo PDF de entrada.
    """

    if not os.path.exists(input_pdf_path):
        print(f"ERRO: Arquivo não encontrado em {input_pdf_path}. Pulando análise.")
        return

    try:
        # Abre o arquivo PDF em modo binário de leitura

        with open(input_pdf_path, 'rb') as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            num_pages = len(pdf_reader.pages)
            
            print("="*50)
            print(f"ANÁLISE DO ARQUIVO: {input_pdf_path}")
            print(f"Número total de páginas: {num_pages}")
            print("="*50)

            for i in range(num_pages):

                # O PyPDF2 v3+ usa .pages para acessar as páginas

                page_content = pdf_reader.pages[i].extract_text()
                
                # Tratamento de conteúdo None (ocorre se a página for apenas imagem/escaneada)

                if page_content is None:
                    print(f'Página {i + 1}: Não foi possível extrair texto (pode ser uma imagem).')
                    word_count = 0
                else:
                    word_count = len(page_content.split())
                    print(f'Página {i + 1} - Número de palavras: {word_count}')
                    
                    # Salvando o conteúdo da página em um arquivo de texto

                    output_txt_path = f'pagina_em_texto-{i + 1}.txt'
                    with open(output_txt_path, 'w', encoding='utf-8') as arquivo_texto:
                        arquivo_texto.write(page_content)
                    print(f" -> Conteúdo da página salvo em '{output_txt_path}'")
            
            print("\nExtração e análise concluídas com sucesso!")

    except PyPDF2.errors.PdfReadError:
        print(f"\nERRO: O arquivo {input_pdf_path} não é um PDF válido ou está corrompido.")
    except Exception as e:
        print(f"\nOcorreu um erro inesperado durante a extração: {e}")


def merge_pdfs(input_pdf_paths, output_pdf_path):

    """
    Unir lista de arquivos PDF em um único arquivo.

    Args:
        input_pdf_paths (list): Uma lista de caminhos (strings) para os arquivos PDF a serem unidos.
        output_pdf_path (str): O caminho para o novo arquivo PDF que será criado.
    """

    pdf_merger = PyPDF2.PdfWriter()
    
    # 1. Adicionar os PDFs de entrada

    print("\n" + "="*50)

    print("INICIANDO FUSÃO DE ARQUIVOS")

    print("="*50)

    for path in input_pdf_paths:
        if os.path.exists(path):
            try:
                # O .append() pode receber o caminho do arquivo diretamente ou um objeto de arquivo

                pdf_merger.append(path)
                print(f"Adicionado: {path}")
            except Exception as e:
                print(f"AVISO: Não foi possível adicionar {path}. Erro: {e}")
        else:
            print(f"AVISO: Arquivo não encontrado: {path}. Pulando.")

    # 2. Escrever o PDF de saída

    if len(pdf_merger.pages) > 0:
        try:
            with open(output_pdf_path, 'wb') as output_file:
                pdf_merger.write(output_file)
            print(f"\nSucesso! Os PDFs foram unidos em: {output_pdf_path}")
        except Exception as e:
            print(f"ERRO ao escrever o arquivo mesclado: {e}")
    else:
        print("\nFusão cancelada. Nenhum PDF válido foi adicionado.")
    

def convert_image_to_pdf(input_image_path, output_pdf_path):

    """
    Converter um arquivo de imagem (PNG ou JPG) em um arquivo PDF.

    Args:
        input_image_path (str): O caminho para o arquivo de imagem de entrada.
        output_pdf_path (str): O caminho para o arquivo PDF de saída.
    """

    if not os.path.exists(input_image_path):
        print(f"ERRO: Arquivo de imagem não encontrado em {input_image_path}. Pulando conversão.")
        return

    try:
        image = Image.open(input_image_path)


        image.save(output_pdf_path, "PDF", resolution=100.0)  # O argumento melhora a qualidade do PDF
        print("\n" + "="*50)
        print("CONVERSÃO DE IMAGEM")
        print("="*50)
        print(f"Sucesso! Imagem '{input_image_path}' convertida para PDF: '{output_pdf_path}'")
        
        # Leitura de verificação

        with open(output_pdf_path, 'rb') as pdf_file:
             reader = PyPDF2.PdfReader(pdf_file)
             print(f"Verificação: O novo PDF possui {len(reader.pages)} página(s).")

    except FileNotFoundError:
        print(f"\nERRO: Arquivo de imagem não encontrado: {input_image_path}")
    except Exception as e:
        print(f"\nOcorreu um erro durante a conversão da imagem: {e}")


def main():

    """Função principal para demonstrar o uso das funções de PDF."""
    
    # Certificando-se de que estes arquivos existam para evitar erros.

    # Testar a estrutura do código
    
    #  1- Análise e Extração 

    extract_and_analyze_pdf(PDF_MASTER)
    
    # 2. União de PDFs 

    pdf_list = [PDF_MASTER, PDF_DOCUMENTO1]
    merge_pdfs(pdf_list, PDF_MERGED_OUTPUT)
    
    # 3. Conversão de Imagem para PDF 

    convert_image_to_pdf(IMAGE_INPUT, PDF_IMAGE_OUTPUT)
    

if __name__ == "__main__":
    # Este bloco garante que a função 'main' será executada quando o script for iniciado
    # e permite que o script seja importado em outros lugares sem rodar o main automaticamente.
    # É uma boa prática em Python.
    main()

ANÁLISE DO ARQUIVO: Master_D.pdf
Número total de páginas: 2
Página 1 - Número de palavras: 131
 -> Conteúdo da página salvo em 'pagina_em_texto-1.txt'
Página 2 - Número de palavras: 6
 -> Conteúdo da página salvo em 'pagina_em_texto-2.txt'

Extração e análise concluídas com sucesso!

INICIANDO FUSÃO DE ARQUIVOS
Adicionado: Master_D.pdf
Adicionado: Documento1.pdf

Sucesso! Os PDFs foram unidos em: Merge.pdf

CONVERSÃO DE IMAGEM
Sucesso! Imagem 'master_img.png' convertida para PDF: 'imagem.pdf'
Verificação: O novo PDF possui 1 página(s).


In [ ]:
#Criação de novo arquivo Pdf

from reportlab.pdfgen import canvas

c = canvas.Canvas("Texto_criado.pdf")
c.drawString(100, 750, "Saudações a todo o pessoal da Master D,")
c.drawString(100, 735, "em especial ao Orientador André!")
c.drawString(100, 720, "isto é desmonstração de correção e melhoria do código, aos, 08/11/2025")



c.save()




In [ ]:
#PDF adicionar marca d'agua

from PyPDF2 import PdfReader, PdfWriter

reader = PdfReader('master_img.pdf')
image_page = reader.pages[0]
writer = PdfWriter()
reader = PdfReader('Master_D.pdf')

for i in range (0, len(reader.pages)):
    content_page = reader.pages[i]
    
    mediabox = content_page.mediabox
    content_page.merge_page(image_page)
    
    
    writer.add_page(content_page)
    
    
with open('Master_D.pdf', 'wb') as pdf:
    writer.write(pdf)